In [31]:
import ISO_Networksii as ISON
import ISO_Networks as ISON1
import datawrangler as dtwrnglr
import re
import operator
import string
from collections import Counter
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

# Load Data

In [2]:
#CMU, FILM2.0, TVPOP
U,F,T = ISON.load_universe()

In [3]:
#Genre sorted function clears up issues with genre labels and repeated instances of screenplays
F1=ISON.genre_sorted_data(F)

[('Action', 290), ('Comedy', 265), ('Drama', 254), ('Crime', 96), ('Adventure', 61), ('Horror', 56), ('Mystery', 12), ('Animation', 10), ('Fantasy', 6), ('Thriller', 6), ('Sci-Fi', 5), ('Biography', 3), ('Romance', 2), ('Western', 2), ('Family', 1), ('Short', 1)]
['Action', 'Comedy', 'Drama', 'Crime', 'Adventure', 'Horror']


/Users/ViVeri/Desktop/ISO_Networks/ISO_Networksii.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  new['documents'] = [film_strip(i) for i in new.text.tolist()]


In [289]:
data=F1.documents.tolist()
labels = F1['labels'].tolist()
print('The genre distribution is:',Counter(labels))

The genre distribution is: Counter({'Action': 290, 'Comedy': 265, 'Drama': 254, 'Crime': 96, 'Adventure': 61, 'Horror': 56, 'Other': 48})


In [38]:
#data has to be handled differently in order to create document themes
data_topics = [i.split() for i in data]

# Vector Space BoW

# Different N-gram vectorizers

In [202]:
cv, data_N=ISON.BoW_ngram(data,1)
# tf, data_T = ISON.BoW_tfidf(data,1)

In [15]:
%time cv2, data_N2=ISON.BoW_ngram(data,2)
%time tf2, data_T2 = ISON.BoW_tfidf(data,2)

CPU times: user 1min 12s, sys: 4.38 s, total: 1min 17s
Wall time: 1min 18s
CPU times: user 1min 24s, sys: 5.34 s, total: 1min 29s
Wall time: 1min 31s


In [18]:
%time cv3, data_N3=ISON.BoW_ngram(data,3)
%time tf3, data_T3 = ISON.BoW_tfidf(data,3)

CPU times: user 2min 57s, sys: 31 s, total: 3min 28s
Wall time: 3min 37s
CPU times: user 3min 24s, sys: 38 s, total: 4min 2s
Wall time: 4min 23s


# Machine Learning- Classification

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RandomizedLogisticRegression
from sklearn import preprocessing
from sklearn import svm, linear_model
from sklearn.linear_model import SGDClassifier

In [9]:
from sklearn.model_selection import cross_val_score
def train_test(data, labels, test_size):
    X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                    stratify=labels, 
                                                    test_size=test_size)
    return X_train, X_test, y_train, y_test

def ISON_ngram_Film_experiment(universe_representation, labels, test_size):
    
    
    #STEP 1: SPLIT THE DATA
    X_train, X_test, y_train, y_test=train_test(universe_representation,labels,test_size)
    print('START')
    
    #STEP 2 : INTIALIZE LOGISTIC REGRESSSION
    model_ML = Pipeline([
    ('clf',LogisticRegression(solver='sag')) 
                                                ])
    print('Fitting corpus into LogReg() model')
    print('Corpus is', str(universe_representation.shape))
    
    #STEP 3: TRAIN MODEL
    clf=model_ML.fit(X_train, y_train)
    
    #STEP 4: PREDICT TEST
    y_pred = clf.predict(X_test)
    print('\n')
    
    #STEP 5: CLASSIFACTION REPORT 
    report =classification_report( y_test, y_pred)
    print(report)
    
    print((y_pred == y_test).mean())
#     print(report_classfication(report)   ) 

    #STEP 6: CROSS VALIDATION SCORE 
    scores=cross_val_score(model_ML, X_train, y_train, cv=10)
    print(scores)
    print(np.mean(scores))
    
    return clf



def ISON_Film_experiment2(universe_representation, labels, test_size):
    ##This one runs Randomized LogRegression unlike the first, takes a bit longer 
    
    #STEP 1: SPLIT THE DATA
    X_train, X_test, y_train, y_test=train_test(universe_representation,labels,test_size)
    print('START')
    
    #STEP 2: 
    RLR = RandomizedLogisticRegression()
    le = preprocessing.LabelEncoder()
    label_encoded=le.fit_transform(labels)
    randomizedLRM=RLR.fit(universe_representation, label_encoded)
    
    
    feature_scores=randomizedLRM.all_scores_
    index = np.where(feature_scores!= 0)[0]
    
    model_ML = Pipeline([
    ('clf',LogisticRegression(solver='sag'))
                                                                    ])
    
    print('Fitting model')
    clf=model_ML.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('\n')
    print(classification_report( y_test, y_pred))
    print((y_pred == y_test).mean())
    
    print('%s features were of importance, the following are the weights:' %len(index))
    print(feature_scores[index])
    print('#'*33, '\n')
    
    for i in index:
        print(i )

    
    print('#'*33, '\n')

# Experimental Results

In [10]:
# Logistic Regression with unigrams
# %time ISON_ngram_Film_experiment(data_N,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 121629)


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




             precision    recall  f1-score   support

     Action       0.70      0.84      0.77        58
  Adventure       0.50      0.08      0.14        12
     Comedy       0.63      0.79      0.70        53
      Crime       0.60      0.32      0.41        19
      Drama       0.55      0.61      0.58        51
     Horror       0.67      0.55      0.60        11
      Other       0.00      0.00      0.00        10

avg / total       0.59      0.63      0.59       214

0.6308411214953271


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.p

[0.62921348 0.58426966 0.59770115 0.59302326 0.58139535 0.57647059
 0.55294118 0.55952381 0.59036145 0.6097561 ]
0.5874656017717665
CPU times: user 4min 58s, sys: 1.44 s, total: 5min
Wall time: 5min 2s


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [14]:
# Logistic regression with tfidif unigrams
# %time ISON_ngram_Film_experiment(data_T,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 121629)


             precision    recall  f1-score   support

     Action       0.39      0.91      0.55        58
  Adventure       0.00      0.00      0.00        12
     Comedy       0.56      0.60      0.58        53
      Crime       0.00      0.00      0.00        19
      Drama       0.45      0.20      0.27        51
     Horror       0.00      0.00      0.00        11
      Other       0.00      0.00      0.00        10

avg / total       0.35      0.44      0.36       214

0.4439252336448598


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.4494382  0.43820225 0.4137931  0.45348837 0.40697674 0.36470588
 0.45882353 0.47619048 0.40963855 0.45121951]
0.43224766235327194
CPU times: user 56.2 s, sys: 405 ms, total: 56.6 s
Wall time: 57 s


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [16]:
#LogRegression with countvectorizer n = (1,2)
%time ISON_ngram_Film_experiment(data_N2,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 5377617)


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




             precision    recall  f1-score   support

     Action       0.69      0.83      0.75        58
  Adventure       0.00      0.00      0.00        12
     Comedy       0.54      0.68      0.60        53
      Crime       0.11      0.05      0.07        19
      Drama       0.45      0.55      0.50        51
     Horror       0.67      0.18      0.29        11
      Other       1.00      0.10      0.18        10

avg / total       0.52      0.54      0.50       214

0.5420560747663551


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_ite

[0.48314607 0.66292135 0.6091954  0.61627907 0.61627907 0.57647059
 0.6        0.61904762 0.61445783 0.63414634]
0.60319433376357
CPU times: user 1h 27min 45s, sys: 1min 12s, total: 1h 28min 57s
Wall time: 1h 36min 36s


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [17]:
#LogRegression with tfidf n = (1,2)
%time ISON_ngram_Film_experiment(data_T2,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 5377617)


             precision    recall  f1-score   support

     Action       0.42      0.88      0.57        58
  Adventure       0.00      0.00      0.00        12
     Comedy       0.44      0.55      0.49        53
      Crime       0.00      0.00      0.00        19
      Drama       0.38      0.20      0.26        51
     Horror       0.00      0.00      0.00        11
      Other       0.00      0.00      0.00        10

avg / total       0.31      0.42      0.34       214

0.4205607476635514


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.41573034 0.3258427  0.51724138 0.5        0.43023256 0.44705882
 0.41176471 0.44047619 0.44578313 0.42682927]
0.4360959091868503
CPU times: user 13min 54s, sys: 10.6 s, total: 14min 4s
Wall time: 14min 9s


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [19]:
%time ISON_ngram_Film_experiment(data_N3,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 13964245)


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




             precision    recall  f1-score   support

     Action       0.61      0.78      0.68        58
  Adventure       0.00      0.00      0.00        12
     Comedy       0.54      0.72      0.61        53
      Crime       0.36      0.21      0.27        19
      Drama       0.48      0.45      0.46        51
     Horror       0.62      0.45      0.53        11
      Other       0.00      0.00      0.00        10

avg / total       0.48      0.54      0.50       214

0.5373831775700935


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.p

[0.52808989 0.58426966 0.52873563 0.56976744 0.65116279 0.61176471
 0.62352941 0.58333333 0.63855422 0.63414634]
0.5953353424615122
CPU times: user 2h 37min 20s, sys: 2min 33s, total: 2h 39min 54s
Wall time: 3h 5min 19s


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [20]:
%time ISON_ngram_Film_experiment(data_T3,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 13964245)


             precision    recall  f1-score   support

     Action       0.44      0.93      0.59        58
  Adventure       0.00      0.00      0.00        12
     Comedy       0.47      0.64      0.54        53
      Crime       0.00      0.00      0.00        19
      Drama       0.59      0.20      0.29        51
     Horror       0.00      0.00      0.00        11
      Other       0.00      0.00      0.00        10

avg / total       0.37      0.46      0.36       214

0.45794392523364486


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.43820225 0.41573034 0.45977011 0.45348837 0.44186047 0.43529412
 0.38823529 0.44047619 0.42168675 0.40243902]
0.4297182910040586
CPU times: user 26min 15s, sys: 35.8 s, total: 26min 51s
Wall time: 27min 56s


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

# Trying to get most important Features
Can only seem to get important features for entire classifier, haven't figure out how to obtain features for each class

In [21]:
%time ISON_Film_experiment2(data_N,labels,.20)

START


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Fitting model


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




             precision    recall  f1-score   support

     Action       0.73      0.83      0.77        58
  Adventure       0.00      0.00      0.00        12
     Comedy       0.63      0.79      0.70        53
      Crime       0.50      0.26      0.34        19
      Drama       0.54      0.63      0.58        51
     Horror       0.75      0.55      0.63        11
      Other       0.00      0.00      0.00        10

avg / total       0.56      0.62      0.58       214

0.6214953271028038
4831 features were of importance, the following are the weights:
[[0.06 ]
 [0.025]
 [0.01 ]
 ...
 [0.02 ]
 [0.01 ]
 [0.04 ]]
################################# 

43
84
87
105
119
177
187
188
196
263
331
333
392
429
494
527
530
576
590
593
598
602
606
615
627
637
675
687
693
734
755
764
779
783
786
805
817
840
855
856
861
866
884
914
975
977
1016
1069
1079
1128
1137
1194
1268
1269
1286
1287
1320
1388
1413
1460
1461
1465
1472
1473
1487
1503
1518
1520
1526
1539
1564
1570
1583
1638
1647
1681
1702
17

55786
55820
55826
55873
55898
55981
56046
56137
56212
56265
56279
56338
56367
56393
56397
56400
56409
56471
56514
56529
56559
56563
56627
56628
56668
56693
56699
56889
56944
56995
57038
57090
57192
57195
57289
57414
57444
57456
57538
57577
57653
57677
57693
57706
57758
57764
57794
57802
57838
57839
57842
57958
57968
57978
57988
57992
58009
58116
58192
58261
58263
58273
58277
58281
58286
58298
58318
58322
58325
58331
58368
58412
58500
58538
58815
58824
58836
59084
59107
59113
59125
59155
59199
59226
59256
59273
59285
59295
59303
59389
59407
59424
59427
59460
59473
59477
59495
59498
59512
59566
59603
59611
59640
59658
59713
59761
59765
59804
59813
59814
59840
59862
59904
59916
59919
59920
59926
59930
59933
59939
59942
59957
60002
60040
60052
60084
60099
60142
60152
60159
60174
60184
60204
60210
60211
60262
60269
60270
60327
60335
60353
60466
60470
60485
60501
60513
60529
60540
60645
60714
60735
60747
60755
60776
60781
60782
60783
60788
60801
60855
60871
60874
60893
60900
60951
60963
6100

99285
99291
99340
99360
99374
99398
99413
99418
99425
99447
99454
99465
99471
99510
99585
99624
99642
99645
99664
99672
99679
99693
99711
99750
99753
99757
99779
99805
99821
99868
99874
99990
100032
100054
100100
100118
100127
100143
100226
100258
100307
100338
100348
100388
100393
100400
100446
100468
100492
100550
100558
100600
100637
100671
100675
100695
100724
100730
100732
100741
100753
100784
100797
100800
100852
100853
100857
100877
100899
100903
100940
100972
100976
101032
101050
101102
101108
101118
101148
101226
101245
101247
101290
101331
101428
101449
101464
101478
101479
101483
101521
101526
101598
101632
101653
101724
101760
101779
101856
101900
101912
101931
101948
102016
102033
102040
102049
102073
102076
102078
102084
102094
102098
102099
102110
102173
102198
102204
102207
102219
102224
102231
102245
102294
102352
102361
102375
102518
102533
102572
102621
102625
102647
102661
102680
102689
102709
102725
102726
102728
102747
102765
102806
102828
102875
102877
102881
102

In [22]:
%time ISON_Film_experiment2(data_T,labels,.20)

START


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class RandomizedLogisticRegression is deprecated; The class RandomizedLogisticRegression is deprecated in 0.19 and will be removed in 0.21.
  warnings.warn(msg, category=DeprecationWarning)


Fitting model


             precision    recall  f1-score   support

     Action       0.41      0.91      0.57        58
  Adventure       0.00      0.00      0.00        12
     Comedy       0.49      0.53      0.51        53
      Crime       0.00      0.00      0.00        19
      Drama       0.32      0.18      0.23        51
     Horror       0.00      0.00      0.00        11
      Other       0.00      0.00      0.00        10

avg / total       0.31      0.42      0.33       214

0.4205607476635514
5021 features were of importance, the following are the weights:
[[0.045]
 [0.22 ]
 [0.01 ]
 ...
 [0.17 ]
 [0.005]
 [0.03 ]]
################################# 

43
84
87
88
105
119
177
263
331
333
392
485
527
586
590
593
598
602
627
637
693
712
733
755
774
775
783
786
805
807
817
832
861
884
899
930
931
948
975
1016
1069
1071
1079
1100
1157
1193
1227
1268
1269
1285
1320
1337
1388
1417
1422
1437
1460
1463
1465
1479
1503
1526
1557
1566
1590
1618
1638
1647
1699
1704
1709
1744
1745
17

/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



13974
13976
13980
13991
14008
14028
14037
14038
14134
14181
14207
14217
14245
14313
14332
14353
14357
14416
14437
14459
14487
14492
14528
14589
14601
14623
14729
14741
14767
14768
14796
14801
14805
14809
14815
14839
14840
14853
14856
14860
14861
14904
14908
14909
14971
14974
14999
15026
15042
15046
15047
15052
15055
15062
15063
15067
15121
15124
15131
15141
15170
15240
15263
15273
15275
15284
15303
15322
15378
15380
15456
15506
15539
15553
15557
15585
15599
15622
15664
15679
15751
15775
15797
15825
15857
15889
15923
15948
15953
15965
15985
15988
16000
16010
16059
16133
16197
16202
16206
16295
16302
16305
16332
16373
16376
16378
16390
16394
16445
16453
16473
16493
16528
16549
16555
16569
16592
16677
16691
16700
16705
16711
16713
16716
16720
16787
16792
16800
16869
16924
16981
16994
16996
17050
17080
17083
17114
17266
17268
17278
17311
17312
17313
17355
17356
17410
17411
17501
17502
17534
17565
17625
17630
17640
17668
17693
17698
17705
17716
17729
17731
17732
17742
17790
17808
17816
178

47496
47510
47517
47541
47565
47566
47585
47595
47603
47606
47608
47614
47661
47672
47699
47711
47753
47768
47836
47849
47852
47861
47896
47902
47946
47954
47958
47972
48002
48073
48074
48094
48096
48116
48129
48211
48219
48316
48317
48325
48330
48359
48381
48397
48407
48413
48419
48450
48474
48487
48489
48492
48493
48498
48519
48526
48571
48574
48651
48671
48685
48690
48698
48702
48754
48756
48757
48779
48800
48832
48847
48865
48895
48976
48987
49033
49092
49095
49120
49127
49347
49350
49351
49391
49396
49408
49416
49427
49534
49558
49578
49652
49667
49696
49714
49766
49769
49799
49820
49821
49829
49868
49914
49921
49925
49935
49967
49979
49996
50002
50003
50032
50035
50050
50062
50065
50066
50085
50101
50109
50117
50136
50140
50186
50222
50255
50256
50266
50281
50297
50304
50312
50347
50365
50412
50413
50423
50434
50474
50487
50533
50557
50559
50572
50584
50612
50619
50629
50648
50653
50657
50677
50716
50762
50786
50827
50838
50866
50897
50935
50964
50973
50983
51011
51022
51025
5103

87406
87422
87458
87494
87497
87508
87522
87572
87655
87669
87704
87705
87708
87739
87758
87761
87765
87774
87777
87785
87798
87805
87820
87834
87841
87846
87853
87854
87864
87901
87908
87913
87931
87943
87947
87994
87996
88003
88013
88057
88063
88085
88093
88101
88111
88117
88149
88173
88208
88217
88260
88263
88309
88332
88345
88368
88370
88380
88422
88441
88461
88500
88505
88508
88515
88553
88579
88616
88618
88638
88672
88675
88738
88742
88770
88773
88774
88780
88793
88862
88887
88927
88933
88948
88969
88972
89044
89047
89065
89080
89086
89108
89114
89117
89124
89139
89257
89388
89403
89411
89414
89433
89434
89510
89568
89628
89638
89650
89672
89685
89711
89712
89732
89734
89745
89776
89794
89810
89828
89833
89877
89911
89914
89915
90015
90028
90063
90076
90096
90100
90104
90109
90132
90155
90191
90350
90369
90371
90374
90411
90416
90440
90471
90491
90496
90598
90603
90849
90867
90890
90911
90955
90981
90986
90999
91058
91060
91075
91103
91120
91144
91173
91211
91286
91364
91396
9143

In [86]:
from sklearn.decomposition import TruncatedSVD
n_components = 10
pca = TruncatedSVD(n_components)
%time pca_data_N = pca.fit_transform(qq)

CPU times: user 11.9 s, sys: 2.61 s, total: 14.5 s
Wall time: 8.34 s


In [29]:
%time ISON_ngram_Film_experiment(pca_data_N,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 20)


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




             precision    recall  f1-score   support

     Action       0.62      0.76      0.68        58
  Adventure       0.33      0.08      0.13        12
     Comedy       0.48      0.57      0.52        53
      Crime       0.00      0.00      0.00        19
      Drama       0.49      0.67      0.57        51
     Horror       0.75      0.27      0.40        11
      Other       0.00      0.00      0.00        10

avg / total       0.46      0.52      0.48       214

0.5233644859813084


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_ite

[0.49438202 0.52808989 0.52873563 0.52325581 0.58139535 0.57647059
 0.44705882 0.54761905 0.63855422 0.51219512]
0.5377756503289408
CPU times: user 2.96 s, sys: 66.7 ms, total: 3.03 s
Wall time: 3.07 s


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [70]:
model, dictionary=dtwrnglr.load_model('0FILMGOLD100_.model')

In [71]:
sauce=ISON1.theme_vector(data_topics, model, dictionary)

In [72]:
%time ISON_ngram_Film_experiment(sauce,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 100)


             precision    recall  f1-score   support

     Action       0.28      1.00      0.44        58
  Adventure       0.00      0.00      0.00        12
     Comedy       0.56      0.09      0.16        53
      Crime       0.00      0.00      0.00        19
      Drama       0.00      0.00      0.00        51
     Horror       0.00      0.00      0.00        11
      Other       0.00      0.00      0.00        10

avg / total       0.21      0.29      0.16       214

0.29439252336448596


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.26966292 0.28089888 0.28735632 0.25581395 0.29069767 0.25882353
 0.25882353 0.28571429 0.27710843 0.26829268]
0.27331922086984506
CPU times: user 2.02 s, sys: 29.6 ms, total: 2.05 s
Wall time: 2.09 s


Pipeline(memory=None,
     steps=[('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [82]:
JJ=data_N.toarray()

In [83]:
qq=np.hstack((JJ, sauce))

(1070, 121729)

In [89]:
%time log_clf=ISON_ngram_Film_experiment(data_N,labels,.20)

START
Fitting corpus into LogReg() model
Corpus is (1070, 121629)


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)




             precision    recall  f1-score   support

     Action       0.70      0.76      0.73        58
  Adventure       0.17      0.08      0.11        12
     Comedy       0.65      0.75      0.70        53
      Crime       0.42      0.26      0.32        19
      Drama       0.51      0.63      0.56        51
     Horror       0.75      0.55      0.63        11
      Other       0.00      0.00      0.00        10

avg / total       0.56      0.60      0.57       214

0.5981308411214953


/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/ViVeri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.p

[0.58426966 0.5505618  0.57471264 0.60465116 0.52325581 0.54117647
 0.6        0.61904762 0.62650602 0.56097561]
0.5785156804584842
CPU times: user 5min 14s, sys: 2.42 s, total: 5min 16s
Wall time: 5min 21s


In [286]:
def sample_screenplay(new,vectorizer, clf):
    reel = []
    lines = [line.rstrip('\n') for line in open(new)]
    text = ' '.join(lines)
    document = ISON.film_strip(text)
    reel.append(document)
    
    newVec = CountVectorizer(vocabulary=vectorizer.vocabulary_)
    vector_document=newVec.transform(reel)
    
    prediction=clf.predict(vector_document)
    
    print(new,'is classified as',prediction )

In [287]:
sample_screenplay('thedarkesthour.txt', cv, log_clf)

thedarkesthour.txt is classified as ['Comedy']
